<a href="https://colab.research.google.com/github/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/blob/main/Documentation/InteraktiveCamera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Grundlegende Idee**
Die Idee ist ein Zwei-Kamera-System zu einem Ein-Kamera-System umzugestalten. Dabei soll die erste Kamera auf einen Sprecher und die zweite Kamera auf ein Blatt Papier gerichtet sein. Sobald der Sprecher nun seinen Zeigefinger auf das Papier legt, wird dieses Kamerabild als Bild in Bild (pip) der ersten Kamerasicht angezeigt.


## Benötigte Hardware
Das System soll so einfach wie möglich sein und keine spezielle Hardware benötigen. Das heißt eine Handykamera, welche über Apps wie beispielsweise [Droid Cam](http://www.dev47apps.com/) auf Android einen Kamerafeed an den PC liefern sollen schon ausreichen. Dementsprechend soll es auch möglich sein dieses Handykamerabild innerhalb einer Bildschirmübertragung anzuzeigen, um so gezeigtes auf dem Bildschirm durch beispielsweise Handzeichnungen zu unterstützen.

## Funktionen
Wie bereits genannt wird der Nutzer zwei Bildinputs auswählen und festlegen müssen welche der Inputs das einzublendende Bild enthält. Dem Nutzer soll es möglich sein durch Zeigen mit dem Zeigefinger die Kamera zu Aktivieren und den Zeigefinger als Mittelpunkt des Bildes zu sehen. Durch eine "Raus-Zoom" Geste wie man sie von Smartphones kennt, wird ein digitaler Zoom ausführbar sein.

## Programmcode
Jeglicher Programmcode ist in dem folgende Repository zu finden:
[Medienverarbeitung Gruppe E - Interactive Camera](https://github.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system)

## Vorgehen im Projekt
Das Projekt wird als Projekttagebuch dokumentiert. Hier werden alle Ideen, Ansätze und Ergebnisse festgehalten. Zu Abschluss des Projekts wird das Gesamtergebnis noch einmal gesondert und aufbereitet präsentiert. 


# Projekttagebuch

## Erster Ansatz und Ergebnisse 
*11.11.20*

Zu erst kümmern wir uns um die Erkennung eines zeigenden Zeigefingers. Dafür trainieren wir ein Keras Modell mit Bildern, welche zeigende Zeigefinger (Bild 1), weiße (karrierte, linierte, blanko) Papiere (Bild 2) und andere Gesten wie flache Hände und Ähnliches (Bild 3).

![alt text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/zu_unterscheidende_Objekte.png)

Das damit antrainierte Modell erkennt das gezeigte Bild und sortiert es der Rubrik ein, welche mit der höchsten Konfidenz übereinstimmt.

Die Verarbeitung des eingehenden Videostreams wird durch die Python-Bibliothek "[OpenCV](https://pypi.org/project/opencv-python/)" ermöglicht.

Durch diesen Ansatz konnten bisher erste Erfolge und enstehende Probleme verzeichnet werden

###Probleme
Es bisher Störfaktoren wie eine Computer-Maus (Bild 6) fälschlicherweise als ein Zeigen erkannt. Jedoch war es auch möglich in einer kontrollierten Umgebung einen Zeigefinger (Bild 4) von einem Blatt Papier (Bild 5) zu unterscheiden.
![alt text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/11.11.20/Ergebnisse.png)

